In [135]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from lambdamart import LambdaMART
import lightgbm as lgb

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm
from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

pd.options.mode.chained_assignment = None

## Run model on clean dataset

In [146]:
# traindf = pd.read_pickle("./pickles/clean_training_set_VU_DM")
# testdf = pd.read_pickle("./pickles/clean_test_set_VU_DM")
df = pd.read_pickle("./pickles/df_small_clean.pkl")
df.describe()

# filename = "clean_training"
filename = "clean_small"

df.head(10)

,prop_id,srch_id,position,price_quality,competitor_bool,availability_bools,visited_before,click_bool,booking_bool,PriceBand,promotion_flag,prop_brand_bool,random_bool,prop_location_score1,prop_location_score2,importance
1196703,2990,79994,24,41.320000,0,0,0,0,0,2,0,1,0,4.47,0.1356,0
1196704,4256,79994,6,22.835000,0,0,0,1,1,2,1,1,0,4.62,0.3492,5
1196705,6427,79994,17,29.357500,0,0,0,0,0,2,0,0,0,4.01,0.2228,0
1196706,8289,79994,30,34.250000,0,0,0,0,0,3,0,1,0,4.41,0.2138,0
1196707,13668,79994,21,30.010000,0,0,0,0,0,2,0,1,0,3.14,0.0184,0
1196708,17105,79994,28,31.967500,1,0,0,0,0,2,0,1,0,4.78,0.2654,0
1196709,19743,79994,12,26.095000,0,0,0,0,0,2,0,1,0,4.43,0.2864,0
1196710,21688,79994,7,33.923333,0,0,0,0,0,2,0,0,0,3.40,0.3013,0
1196711,28818,79994,19,39.143333,0,0,0,0,0,2,0,1,0,4.79,0.2752,0
1196712,37093,79994,27,31.315000,1,0,0,0,0,2,1,1,0,4.68,0.3108,0


In [147]:
dfcopy = df.copy()

y = df["position"]
df = df.drop(["position", "prop_id"], axis=1).copy()
X = df

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1, shuffle=False)

In [149]:
query_lengths = []
[query_lengths.append(len(group)) for srchid, group in X_train.groupby('srch_id')]
gbm = lgb.LGBMRanker(label_gain=query_lengths)
gbm

LGBMRanker(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
           importance_type='split',
           label_gain=[33, 26, 6, 31, 29, 32, 21, 33, 33, 31, 33, 34, 31, 24,
                       17, 18, 6, 31, 23, 5, 33, 31, 32, 32, 32, 5, 34, 34, 16,
                       30, ...],
           learning_rate=0.1, max_depth=-1, min_child_samples=20,
           min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
           n_jobs=-1, num_leaves=31, objective=None, random_state=None,
           reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
           subsample_for_bin=200000, subsample_freq=0)

In [150]:
query_train = [X_train.shape[0]]
query_val = [X_val.shape[0]]
query_test = [X_test.shape[0]]

In [151]:
model = gbm.fit(X_train, y_train, group=query_lengths,
        eval_set=[(X_val, y_val)], eval_group=[query_val],
        early_stopping_rounds=10, verbose=True,
        eval_at=[1,2,3,4,5,6,7,8,9,10],
        callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1)])

[1]	valid_0's ndcg@1: 0.96875	valid_0's ndcg@2: 0.992928	valid_0's ndcg@3: 0.987255	valid_0's ndcg@4: 0.973636	valid_0's ndcg@5: 0.952494	valid_0's ndcg@6: 0.907089	valid_0's ndcg@7: 0.841156	valid_0's ndcg@8: 0.833882	valid_0's ndcg@9: 0.843425	valid_0's ndcg@10: 0.855374
Training until validation scores don't improve for 10 rounds
[2]	valid_0's ndcg@1: 0.96875	valid_0's ndcg@2: 0.992928	valid_0's ndcg@3: 0.987255	valid_0's ndcg@4: 0.973636	valid_0's ndcg@5: 0.952494	valid_0's ndcg@6: 0.907089	valid_0's ndcg@7: 0.841156	valid_0's ndcg@8: 0.833882	valid_0's ndcg@9: 0.843425	valid_0's ndcg@10: 0.855374
[3]	valid_0's ndcg@1: 0.96875	valid_0's ndcg@2: 0.992928	valid_0's ndcg@3: 0.987255	valid_0's ndcg@4: 0.973636	valid_0's ndcg@5: 0.952494	valid_0's ndcg@6: 0.907089	valid_0's ndcg@7: 0.841156	valid_0's ndcg@8: 0.833882	valid_0's ndcg@9: 0.843425	valid_0's ndcg@10: 0.855374
[4]	valid_0's ndcg@1: 0.96875	valid_0's ndcg@2: 0.992928	valid_0's ndcg@3: 0.987255	valid_0's ndcg@4: 0.973636	valid_

In [152]:
# X = df.drop("position", axis=1).copy()
predictions = model.predict(X_train)
print(model.feature_importances_)
print(predictions)

[ 5  7  0  0  1  0  0  1  0  0  0 11  5  0]
[-0.05001987 -0.07881249 -0.05001987 ...  0.06133973  0.06133973
 -0.07881249]


In [153]:
X_train["predicted_rank"] = predictions
X_train["position"] = y
X_train["srch_id"] = dfcopy.srch_id
predictions_sorted = []

for srchid, group in X_train.groupby('srch_id'):
    predictions = [x for _,x in sorted(zip(group.predicted_rank, group.position))]
    predictions_sorted.append(predictions)

predictions_sorted = [item for sublist in predictions_sorted for item in sublist]
X_train["predicted_rank"] = predictions_sorted
X_train.to_pickle("./pickles/LGBMranker_small_solution.pkl")

In [128]:
# convert to numpy array
# y = df['prop_id'].to_numpy()
# X = df.drop("prop_id", axis=1).copy().to_numpy()
rfc = RandomForestClassifier(n_jobs=1, n_estimators=100)
model = rfc.fit(X_train, y_train)

In [63]:
@jit
def rand_for(X_train, y_train):
    rfc = RandomForestClassifier(n_jobs=1, n_estimators=100)
    model = rfc.fit(X_train, y_train)
    
#     model = LambdaMART(training_data=X_train, number_of_trees=2, learning_rate=0.1)
#     model.fit()
    return model


# Fit the model on entire trainingset
model = rand_for(X_train.to_numpy(), y_train.to_numpy())

# df = pd.read_pickle("./pickles/df_small_clean.pkl")
# y = df["position"]
# X = df.copy()

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


# Drop position from test set
# X_test.drop("position", axis=1)

# Add empty row to append on later on
# X_test['position'] = None

#df = pd.DataFrame(data=numpy_result, index=df['srch_id'], columns=["prop_id", "srch_id", "position", "price_quality", "competitor_bool", "availability_bools", "visited_before", "click_bool", "booking_bool", "promotion_flag", "prop_brand_bool", "random_bool"])

<ipython-input-63-02a7407d6e88>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "rand_for" failed type inference due to: Untyped global name 'RandomForestClassifier': cannot determine Numba type of <class 'abc.ABCMeta'>

File "<ipython-input-63-02a7407d6e88>", line 3:
def rand_for(X_train, y_train):
    rfc = RandomForestClassifier(n_jobs=1, n_estimators=100)
    ^

  @jit
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "rand_for" was compiled in object mode without forceobj=True.

File "<ipython-input-63-02a7407d6e88>", line 2:
@jit
def rand_for(X_train, y_train):
^

  state.func_ir.loc))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numba/object_mode_passes.py:187: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behavi

ValueError: Number of labels=8 does not match number of samples=3956

In [59]:
df = pd.read_pickle("./pickles/df_small_clean.pkl")
y = df["position"]
X = df.drop("position", axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

query_train = [X_train.shape[0]]
query_test = [X_test.shape[0]]


# For LightGBM data containers
# x, x_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Create the LightGBM data containers
# train_data = lgb.Dataset(x, label=y)
# test_data = lgb.Dataset(x_test, label=y_test)

# gbm = lgb.LGBMRanker()
# gbm.fit(X_train, y_train, group=query_train,
#         eval_set=[(X_train, y_train)], eval_group=[query_train],
#         eval_at=[5, 10, 20], early_stopping_rounds=50)

In [61]:
df = pd.read_pickle("./pickles/df_small_clean.pkl")
y = df["position"]
X = df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [64]:
predictions_column = []
#     model = LambdaMART(group, 10, 0.01, 'sklearn')
#     model.fit()
#     gbm = lgb.LGBMRanker()


# Make predictions per search id (not on entire testset!)
for srchid, group in X_test.groupby('srch_id'):

    y_train = group.position
#     group = group.drop(["position", "srch_id", "prop_id"], axis=1).copy()
    
    # Fit the model on this part of trainingset
    model = rand_for(group.to_numpy(), y_train.to_numpy())
    predictions = list(model.predict(group))
    
    # Rank positions in order of price_quality
    predictions = [x for _,x in sorted(zip(group.PriceBand ,predictions), reverse=True)]
    predictions_column.append(predictions)

# Flatten list to fit entire testset
predictions_column = [item for sublist in predictions_column for item in sublist]
X_test["predicted_rank"] = predictions_column
X_test.head()


X_test.to_pickle("./pickles/random_forest_small_solution.pkl")

In [155]:
X_test.head(10)

,prop_id,srch_id,position,price_quality,competitor_bool,availability_bools,visited_before,click_bool,booking_bool,PriceBand,promotion_flag,prop_brand_bool,random_bool,predicted_rank
1597336,96260,107340,19,23.840000,0,1,0,0,0,6,1,0,0,36
3713106,65606,249343,7,43.980000,0,0,0,0,0,7,0,1,0,30
1645721,7459,110549,31,36.333332,1,0,0,0,0,2,0,1,0,29
4924736,92046,330548,13,19.610001,0,0,0,0,0,2,0,0,0,3
800009,110792,53512,27,27.000000,0,0,0,1,1,4,0,1,0,15
1620007,133007,108848,9,38.305000,0,1,0,0,0,6,0,0,0,8
4503513,101494,302149,9,52.333332,0,1,0,0,0,6,0,0,0,37
2392951,101052,160691,10,63.075001,1,1,0,0,0,7,0,0,0,14
1760153,4345,118200,3,59.666668,0,1,0,0,0,5,0,0,0,19
2582510,35615,173399,3,23.500000,0,0,0,0,0,3,0,1,0,9
